# Replication - pbcseq

Here we provide a notebook to replicate the application to the pbcseq data set from R-Survival

The notebook replicates the results in:

- /out/application/pbcseq.csv

The main script can be found at:

- /scripts/application/pbcseq.ipynb




In [43]:
# google colab specific - installing probcox
!pip3 install probcox

In [44]:
%load_ext rpy2.ipython

The rpy2.ipython extension is already loaded. To reload it, use:
  %reload_ext rpy2.ipython


In [45]:
%%R 
install.packages('survival')
install.packages('glmnet')

R[write to console]: Installing package into ‘/usr/local/lib/R/site-library’
(as ‘lib’ is unspecified)

R[write to console]: trying URL 'https://cran.rstudio.com/src/contrib/survival_3.2-13.tar.gz'

R[write to console]: Content type 'application/x-gzip'
R[write to console]:  length 6545339 bytes (6.2 MB)

R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[w

In [46]:
# Modules
# =======================================================================================================================
import os
import sys
import shutil
import subprocess
import tqdm

import numpy as np
import pandas as pd

import torch
from torch.distributions import constraints

import pyro
import pyro.distributions as dist

from pyro.infer import SVI, Trace_ELBO

import matplotlib.pyplot as plt

import warnings
warnings.filterwarnings("ignore")

import probcox as pcox

dtype = torch.FloatTensor

np.random.seed(9044)
torch.manual_seed(8734)

In [47]:
# Loading Data
# =======================================================================================================================
%%R -o data
library(survival)
data(pbc)
data <- pbcseq

In [48]:
data

,id,futime,status,trt,age,sex,day,ascites,hepato,spiders,edema,bili,chol,albumin,alk.phos,ast,platelet,protime,stage
1,1,400,2,1,58.765229,f,0,1,1,1,1.0,14.5,261,2.60,1718,138.0,190,12.2,4
2,1,400,2,1,58.765229,f,192,1,1,1,1.0,21.3,-2147483648,2.94,1612,6.2,183,11.2,4
3,2,5169,0,1,56.446270,f,0,0,1,1,0.0,1.1,302,4.14,7395,113.5,221,10.6,3
4,2,5169,0,1,56.446270,f,182,0,1,1,0.0,0.8,-2147483648,3.60,2107,139.5,188,11.0,3
5,2,5169,0,1,56.446270,f,365,0,1,1,0.0,1.0,-2147483648,3.55,1711,144.2,161,11.6,3
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1941,312,1457,0,0,33.152635,f,0,0,0,1,0.0,6.4,576,3.79,2115,136.0,200,10.8,2
1942,312,1457,0,0,33.152635,f,206,0,0,0,0.0,5.5,-2147483648,3.20,1678,124.0,189,10.9,2
1943,312,1457,0,0,33.152635,f,390,0,0,0,0.0,7.4,312,3.56,1767,166.0,148,11.7,2
1944,312,1457,0,0,33.152635,f,775,0,0,1,0.5,16.3,688,3.34,2460,173.0,138,13.0,2


In [49]:
np.sum(np.asarray(data['status'] == 2).astype(int))

725

In [50]:
# Long Format
# =======================================================================================================================
surv = np.zeros((0, 3))
X = np.zeros((0, 17))

for idx in np.unique(data['id']):
  dd = data[data['id'] == idx].copy()
  tt = np.asarray(dd['day'])
  tt = np.concatenate((tt, dd['futime'][-1, None]))

  time = np.concatenate((tt[:-1, None], tt[1:, None], np.asarray(dd['status'] == 2).astype(int)[:, None]), axis=1)
  surv = np.concatenate((surv, time))

  # generate relevant covariates
  xx = np.concatenate((np.asarray(dd['trt'] == 1).astype(int)[:, None], 
  np.asarray(dd['sex'] == 'm').astype(int)[:, None],
  np.asarray(dd['edema'] == 0).astype(int)[:, None],
  np.asarray(dd['edema'] == 0.5).astype(int)[:, None],
  np.asarray(dd['stage'] == 2).astype(int)[:, None],
  np.asarray(dd['stage'] == 3).astype(int)[:, None], 
  np.asarray(dd['stage'] == 4).astype(int)[:, None], 
  np.asarray(dd['ascites']).astype(float)[:, None], 
  np.asarray(dd['hepato']).astype(float)[:, None], 
  np.asarray(dd['spiders']).astype(float)[:, None], 
  np.asarray(dd['bili']).astype(float)[:, None], 
  np.asarray(dd['chol']).astype(float)[:, None], 
  np.asarray(dd['albumin']).astype(float)[:, None], 
  np.asarray(dd['alk.phos']).astype(float)[:, None], 
  np.asarray(dd['platelet']).astype(float)[:, None],
  np.asarray(dd['protime']).astype(float)[:, None], 
  np.asarray(dd['age']).astype(float)[:, None]), axis=1)

  # forward fill missing data
  for cc in range(xx.shape[1]):
    for rr in range(1, xx.shape[0]):
      if xx[rr, cc] <= -2000:
        xx[rr, cc] = xx[rr-1, cc]

  X = np.concatenate((X, xx))

# standardize continous variables
X[:, -7:]  = ((X[:, -7:] - np.mean(X[:, -7:], axis=0)) / np.sqrt(np.var(X[:, -7:], axis=0)))

# R dataset 
rd = pd.DataFrame(np.concatenate((surv, X), axis=1))
rd.columns = ['V' + str(ii) for ii in range(1, rd.shape[1]+1)]

In [51]:
# R-Survival - Standard Cox Model
# =======================================================================================================================
%%R -i rd -o r_coef -o r_se
library(survival)
set.seed(7)
Surv(rd$V1, rd$V2, rd$V3)
m = coxph(Surv(V1, V2, V3)~., data=rd)
r_coef = unname(m$coefficients)
r_se = unname(sqrt(diag(m$var)))
summary(m)

Call:
coxph(formula = Surv(V1, V2, V3) ~ ., data = rd)

  n= 1945, number of events= 725 

        coef exp(coef) se(coef)      z Pr(>|z|)    
V4  -0.07423   0.92845  0.07625 -0.974   0.3303    
V5   0.58540   1.79570  0.09953  5.882 4.06e-09 ***
V6  -0.54893   0.57757  0.13619 -4.031 5.56e-05 ***
V7  -0.30095   0.74011  0.12925 -2.329   0.0199 *  
V8  -0.13309   0.87538  0.26985 -0.493   0.6219    
V9  -0.06925   0.93310  0.25368 -0.273   0.7849    
V10  0.04908   1.05030  0.25274  0.194   0.8460    
V11 -0.11937   0.88748  0.12136 -0.984   0.3253    
V12  0.18251   1.20023  0.09273  1.968   0.0490 *  
V13  0.07358   1.07636  0.08338  0.883   0.3775    
V14  0.31348   1.36818  0.03632  8.630  < 2e-16 ***
V15 -0.04931   0.95189  0.03700 -1.333   0.1826    
V16 -0.25986   0.77116  0.04511 -5.760 8.40e-09 ***
V17  0.18142   1.19892  0.03467  5.232 1.67e-07 ***
V18 -0.01134   0.98873  0.04564 -0.248   0.8039    
V19  0.15490   1.16754  0.03145  4.925 8.43e-07 ***
V20  0.28059   1.32390  0

In [52]:
# ProbCox - Standard
# =======================================================================================================================
surv = torch.from_numpy(surv).type(dtype)
X = torch.from_numpy(X).type(dtype)

# Collect sample information for re-weigthing of likelihood
total_obs = surv.shape[0]
total_events = torch.sum(surv[:, -1] == 1).numpy().tolist()
sampling_proportion=[total_obs, None, total_events, None]
print(sampling_proportion)

# set desired level of batchsize
batchsize = 256

# Here we can define a custom linear predictor which is then evalauted corresponding to the Cox Partial Likelihood
def predictor(data):
    theta =  pyro.sample("theta", dist.Normal(loc=0, scale=1).expand([data[1].shape[1], 1])).type(dtype)
    pred = torch.mm(data[1], theta)
    return(pred)

# run inference
sampling_proportion[1] = batchsize
eta=5
run = True
while run:
    run = False
    pyro.clear_param_store()
    m = pcox.PCox(sampling_proportion=sampling_proportion, predictor=predictor)
    m.initialize(eta=eta, num_particles=5)
    loss=[0]
    for ii in tqdm.tqdm(range((25000))):
        # random sub-sampling
        idx = np.random.choice(range(surv.shape[0]), batchsize, replace=False)
        data=[surv[idx], X[idx]]
        loss.append(m.infer(data=data))
        # divergence check
        if loss[-1] != loss[-1]:
            eta = eta * 0.1
            run=True
            break   
g = m.return_guide()
out = g.quantiles([0.025, 0.5, 0.975])

a = np.round(out['theta'][1].detach().numpy()[:, 0], 2)
b = np.round(torch.diag(pyro.get_param_store()['AutoMultivariateNormal.scale_tril']).detach().numpy(), 2)
c =np.sign(out['theta'][0].detach().numpy()) == np.sign(out['theta'][2].detach().numpy())
for ii in range(X.shape[1]):
    if c[ii]:
        sig = '*'
    else:
        sig = ''
    print(str(a[ii]) + sig + ', (' + str(b[ii]) + ')')

CI = pcox.metrics(surv=surv.numpy(), linpred=(torch.mm(X, out['theta'][1]).detach().numpy()))
print(CI.concordance())

[1945, None, 725, None]


100%|██████████| 25000/25000 [03:50<00:00, 108.50it/s]

-0.07, (0.08)
0.58*, (0.1)
-0.55*, (0.14)
-0.3*, (0.08)
-0.12, (0.24)
-0.03, (0.13)
0.08, (0.09)
-0.09, (0.11)
0.2*, (0.09)
0.08, (0.08)
0.3*, (0.03)
-0.05, (0.04)
-0.25*, (0.04)
0.18*, (0.03)
0.03, (0.06)
0.15*, (0.03)
0.28*, (0.04)


0.728871787824653


In [53]:
# Make High-dimensional Data
# =======================================================================================================================
X = torch.cat((X, torch.normal(0, 1, (surv.shape[0], 2000))), axis=1)

# R dataset 
rd = pd.DataFrame(np.concatenate((surv.detach().numpy(), X.detach().numpy()), axis=1))
rd.columns = ['V' + str(ii) for ii in range(1, rd.shape[1]+1)]

In [54]:
# R-glmnet - Adaptive Lasso  - Cox Model
# =======================================================================================================================
%%R -i rd -o r_coef_alasso
set.seed(13)
library(glmnet)
library(survival)

yss = Surv(rd$V1, rd$V2, rd$V3)
cv.fit <-cv.glmnet(as.matrix(rd[, 4:2020]), yss, family ="cox", nfolds=5, parallel=FALSE, type.measure ="C")
w <- 1/abs(as.numeric(coef(cv.fit, s=cv.fit$lambda.1se)))
w[w == Inf] <- 1000000 
cv.fit <-cv.glmnet(as.matrix(rd[, 4:2020]), yss, family ="cox", nfolds=5, parallel=FALSE, type.measure="C", penalty.factor=w)
r_coef_alasso <- as.matrix(unname(coef(cv.fit, s=cv.fit$lambda.min)))


In [55]:
CI = pcox.metrics(surv=surv.numpy(), linpred=(torch.mm(X, torch.from_numpy(r_coef_alasso).type(dtype)).detach().numpy()))
print(CI.concordance())

0.718882004648356


In [56]:
print('# identified as non-0 paramters: ', np.sum(r_coef_alasso != 0))

# identified as non-0 paramters:  6


In [57]:
# ProbCox - High-dimensional
# =======================================================================================================================
def predictor(data):
    theta =  pyro.sample("theta", dist.StudentT(1, loc=0, scale=0.001).expand([data[1].shape[1], 1])).type(dtype)
    pred = torch.mm(data[1], theta)
    return(pred)

# run inference
eta=5
run = True
while run:
    run = False
    pyro.clear_param_store()
    m = pcox.PCox(sampling_proportion=sampling_proportion, predictor=predictor)
    m.initialize(eta=eta, rank=20, num_particles=5)
    loss=[0]
    for ii in tqdm.tqdm(range((25000))):
        # random sub-sampling
        idx = np.random.choice(range(surv.shape[0]), batchsize, replace=False)
        data=[surv[idx], X[idx]]
        loss.append(m.infer(data=data))
        # divergence check
        if loss[-1] != loss[-1]:
            eta = eta * 0.1
            run=True
            break   

g = m.return_guide()
out_highdim = g.quantiles([0.025, 0.5, 0.975])

for ii in range(17):
  if np.sign(out_highdim['theta'][0][ii].detach().numpy()) == np.sign(out['theta'][2][ii].detach().numpy()):
    print(out_highdim['theta'][1][ii].detach().numpy()[0], '*')
  else:
    print(out_highdim['theta'][1][ii].detach().numpy()[0], '')

print('# identified as non-0 paramters: ', np.sum(np.sign(out_highdim['theta'][0].detach().numpy()) == np.sign(out_highdim['theta'][2].detach().numpy())))

CI = pcox.metrics(surv=surv.numpy(), linpred=(torch.mm(X, out_highdim['theta'][1]).detach().numpy()))
print(CI.concordance())

100%|██████████| 25000/25000 [10:31<00:00, 39.56it/s]

0.00011517285 
0.53957367 *
-0.29459947 *
-0.0009258727 *
-0.00021118332 
0.0006594502 
-0.00048401792 
4.1510008e-05 
0.17458677 
-0.00043079036 
0.33032498 *
-0.00017172944 
-0.29141164 *
0.15128042 *
0.00012748726 
0.15192679 *
0.2750908 *
# identified as non-0 paramters:  7


0.7413709662964313


In [58]:
print('# identified as non-0 paramters: ', np.sum(np.sign(out_highdim['theta'][0].detach().numpy()) == np.sign(out_highdim['theta'][2].detach().numpy())))

CI = pcox.metrics(surv=surv.numpy(), linpred=(torch.mm(X, out_highdim['theta'][1]).detach().numpy()))
print(CI.concordance())

# identified as non-0 paramters:  7
0.7413709662964313


In [59]:
# Summary Table
# =======================================================================================================================
def summarize(est, lower, upper, u=True):
  if u:
    decision = np.sign(lower) == np.sign(upper)
  else:
    decision = np.repeat(False, 17)
  ll = []
  ll2 = []
  for ii in range(17):
    helpvar = np.round(est, 2)
    if decision[ii]:
      ll.append(str(helpvar[ii]) + '*')
    else: 
      ll.append(str(helpvar[ii]))
    if u:
      ll2.append('[' + str(np.round(lower[ii], 2)) + ', ' + str(np.round(upper[ii], 2)) + ']')
    else: 
      ll2.append('-')
  return(ll, ll2)

res = pd.DataFrame(np.zeros((20, 12)))
res.iloc[:, :] = ''

res.iloc[:, 0] = ['Var:', 'trt', 'sex', 'edema=0', 'edema=0.5', 'stage=2', 'stage=3', 'stage=4', 'ascites', 'hepato', 'spiders', 'bili', 'chol', 'albumin', 'alk.phos', 'platelet', 'protime', 'age', '', 'HarrelsC']


# ProbCox - Standard
a, b = summarize(est=out['theta'][1].detach().numpy()[:, 0], lower=out['theta'][0].detach().numpy()[:, 0], upper=out['theta'][2].detach().numpy()[:, 0])
res.iloc[0, 1] = '$\hat{\theta}$'
res.iloc[0, 2] = '$HPD_{95\%}$'
res.iloc[1:18, 1] = a
res.iloc[1:18, 2] = b
res.iloc[19, 1] = '0.729'

# R-Survival - Standard
a, b = summarize(est=r_coef, lower=r_coef - 1.96*r_se, upper=r_coef + 1.96*r_se)
res.iloc[0, 4] = '$\hat{\theta}$'
res.iloc[0, 5] = '$CI_{95\%}$'
res.iloc[1:18, 4] = a
res.iloc[1:18, 5] = b
res.iloc[19, 4] = '0.729'

# ProbCox - High-dimensional
a, b = summarize(est=out_highdim['theta'][1].detach().numpy()[:, 0], lower=out_highdim['theta'][0].detach().numpy()[:, 0], upper=out_highdim['theta'][2].detach().numpy()[:, 0])
res.iloc[0, 7] = '$\hat{\theta}$'
res.iloc[0, 8] = '$HPD_{95\%}$'
res.iloc[1:18, 7] = np.asarray(a)[:17]
res.iloc[1:18, 8] = np.asarray(b)[:17]
res.iloc[19, 7] = '0.741'

# R-glmnet - High-dimensional
a, b = summarize(est=r_coef_alasso, lower=None, upper=None, u=False)
res.iloc[0, 10] = '$\hat{\theta}$'
res.iloc[0, 11] = '$CI_{95\%}$'
res.iloc[1:18, 10] = a
res.iloc[1:18, 11] = b
res.iloc[19, 10] = '0.726'
 
res

,0,1,2,3,4,5,6,7,8,9,10,11
0,Var:,$\hat{\theta}$,$HPD_{95\%}$,,$\hat{\theta}$,$CI_{95\%}$,,$\hat{\theta}$,$HPD_{95\%}$,,$\hat{\theta}$,$CI_{95\%}$
1,trt,-0.07,"[-0.22, 0.08]",,-0.07,"[-0.22, 0.08]",,0.0,"[-0.0, 0.0]",,[0.],-
2,sex,0.58*,"[0.39, 0.78]",,0.59*,"[0.39, 0.78]",,0.54*,"[0.34, 0.74]",,[0.43],-
3,edema=0,-0.55*,"[-0.82, -0.28]",,-0.55*,"[-0.82, -0.28]",,-0.29*,"[-0.48, -0.11]",,[-0.31],-
4,edema=0.5,-0.3*,"[-0.47, -0.13]",,-0.3*,"[-0.55, -0.05]",,-0.0,"[-0.0, 0.0]",,[0.],-
5,stage=2,-0.12,"[-0.59, 0.35]",,-0.13,"[-0.66, 0.4]",,-0.0,"[-0.0, 0.0]",,[0.],-
6,stage=3,-0.03,"[-0.29, 0.23]",,-0.07,"[-0.57, 0.43]",,0.0,"[-0.0, 0.0]",,[0.],-
7,stage=4,0.08,"[-0.09, 0.25]",,0.05,"[-0.45, 0.54]",,-0.0,"[-0.0, 0.0]",,[0.],-
8,ascites,-0.09,"[-0.31, 0.13]",,-0.12,"[-0.36, 0.12]",,0.0,"[-0.0, 0.0]",,[0.],-
9,hepato,0.2*,"[0.03, 0.37]",,0.18*,"[0.0, 0.36]",,0.17,"[-0.03, 0.38]",,[0.],-


In [60]:
res.iloc[:, 2] = res.iloc[:, 2].apply(lambda x: x.replace("[", "("))
res.iloc[:, 2] = res.iloc[:, 2].apply(lambda x: x.replace("]", ")"))

res.iloc[:, 5] = res.iloc[:, 5].apply(lambda x: x.replace("[", "("))
res.iloc[:, 5] = res.iloc[:, 5].apply(lambda x: x.replace("]", ")"))

res.iloc[:, 8] = res.iloc[:, 8].apply(lambda x: x.replace("[", "("))
res.iloc[:, 8] = res.iloc[:, 8].apply(lambda x: x.replace("]", ")"))


res.iloc[:, 10] = res.iloc[:, 10].apply(lambda x: x.replace("[", ""))
res.iloc[:, 10] = res.iloc[:, 10].apply(lambda x: x.replace("]", ""))

In [61]:
res

,0,1,2,3,4,5,6,7,8,9,10,11
0,Var:,$\hat{\theta}$,$HPD_{95\%}$,,$\hat{\theta}$,$CI_{95\%}$,,$\hat{\theta}$,$HPD_{95\%}$,,$\hat{\theta}$,$CI_{95\%}$
1,trt,-0.07,"(-0.22, 0.08)",,-0.07,"(-0.22, 0.08)",,0.0,"(-0.0, 0.0)",,0.,-
2,sex,0.58*,"(0.39, 0.78)",,0.59*,"(0.39, 0.78)",,0.54*,"(0.34, 0.74)",,0.43,-
3,edema=0,-0.55*,"(-0.82, -0.28)",,-0.55*,"(-0.82, -0.28)",,-0.29*,"(-0.48, -0.11)",,-0.31,-
4,edema=0.5,-0.3*,"(-0.47, -0.13)",,-0.3*,"(-0.55, -0.05)",,-0.0,"(-0.0, 0.0)",,0.,-
5,stage=2,-0.12,"(-0.59, 0.35)",,-0.13,"(-0.66, 0.4)",,-0.0,"(-0.0, 0.0)",,0.,-
6,stage=3,-0.03,"(-0.29, 0.23)",,-0.07,"(-0.57, 0.43)",,0.0,"(-0.0, 0.0)",,0.,-
7,stage=4,0.08,"(-0.09, 0.25)",,0.05,"(-0.45, 0.54)",,-0.0,"(-0.0, 0.0)",,0.,-
8,ascites,-0.09,"(-0.31, 0.13)",,-0.12,"(-0.36, 0.12)",,0.0,"(-0.0, 0.0)",,0.,-
9,hepato,0.2*,"(0.03, 0.37)",,0.18*,"(0.0, 0.36)",,0.17,"(-0.03, 0.38)",,0.,-


In [62]:
# used to save data to google drive - from which we read data on demand - remove if google Colab is not used
from google.colab import drive
drive.mount('/content/drive')


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [63]:
# set drive to fake data folder
res.to_csv('/content/drive/MyDrive/colab/pbcseq.csv', sep=';')